In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from statsmodels.graphics.regressionplots import abline_plot
import matplotlib
import geopandas
from sklearn.decomposition import PCA

In [2]:
pd.set_option('max_columns', 100)

In [3]:
os.getcwd()

'/Users/alvastrand/Library/Mobile Documents/com~apple~CloudDocs/Documents/OU/Research/notebooks/Casey'

In [4]:
path = '/Volumes/TOSHIBA EXT/Data/Processed/arrival_master_2022-05-13_5000/'

os.chdir(path)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/TOSHIBA EXT/Data/Processed/arrival_master_2022-05-13_5000/'

In [ ]:
os.getcwd()

In [ ]:
filename = 'arrival_master_2022-05-13.csv'

df = pd.read_csv(filename)
print(df.shape)

df.head(2)

In [ ]:
df = df.rename(columns={'cell_lat': 'lat', 'cell_lng': 'lon'})

df = df.sort_values(by=['species', 'cell', 'year'])

df = df.reset_index(drop=True)

print(df.shape)

df.head(2)

In [ ]:
os.getcwd()

In [ ]:
path = '/Users/alvastrand/Library/Mobile Documents/com~apple~CloudDocs/Documents/OU/Research/data/output/'

os.chdir(path)

In [ ]:
os.getcwd()

In [ ]:
filename = 'ncep_narr_air_2m_ebd_US_states_east_Mississippi_0101_0731_complete_zerofilled_grid_cell_ids_2002_2019_relApr-2020_Casey_lats_lons.csv'

df_t = pd.read_csv(filename)
print(df_t.shape)

df_t.head(2)

In [ ]:
df_t = df_t.rename(columns={'gr_mn': 'air_temp_degrees_celsius', 'cell_lat': 'lat', 'cell_lng': 'lon'})

df_t = df_t[['cell', 'lat', 'lon', 'year', 'air_temp_kelvin', 'air_temp_degrees_celsius']]

df_t = df_t.sort_values(by=['cell', 'year'])

df_t = df_t.reset_index(drop=True)

print(df_t.shape)

df_t.head(2)

In [ ]:
df_merged = df.merge(df_t, on=['cell', 'year', 'lat', 'lon'])
print(df_merged.shape)

df_merged.head(2)

In [ ]:
df_merged = df_merged.dropna(subset=['arr_GAM_mean'])
print(df_merged.shape)

df_merged.head(2)

In [ ]:
df_merged = df_merged[df_merged['VALID_GAM'] == True]

df_merged = df_merged.reset_index(drop=True)

print(df_merged.shape)

df_merged.head(2)

In [ ]:
list_sci_names = sorted(df_merged['species'].unique())
print(len(list_sci_names))

In [ ]:
df_speeds = df_merged[['species', 'beta_gamma_mean']].drop_duplicates()

df_speeds = df_speeds.reset_index(drop=True)
print(df_speeds.shape)

df_speeds.head(2)

In [ ]:
df_means = df_merged[['species', 'arr_IAR_mean']].groupby(['species']).mean()

df_means = df_means.reset_index()

df_means = df_means.rename(columns={'arr_IAR_mean': 'mean_arrival_day'})

print(df_means.shape)

df_means.head(2)

In [ ]:
df_speeds_means = df_speeds.merge(df_means, on='species')

df_speeds_means = df_speeds_means.rename(columns={'species': 'scientific_name'})

df_speeds_means['scientific_name'] = df_speeds_means['scientific_name'].str.replace('_', ' ')

print(df_speeds_means.shape)

df_speeds_means.head(2)

In [ ]:
os.getcwd()

In [ ]:
path = '/Users/alvastrand/Downloads/states_21basic/'

os.chdir(path)

In [ ]:
os.getcwd()

In [ ]:
df_usa = geopandas.read_file('states.shp')
print(df_usa.shape)

df_usa.head(2)

In [ ]:
df_usa_c = df_usa[(df_usa['STATE_NAME'] == 'Alabama') | (df_usa['STATE_NAME'] == 'Connecticut') | 
                  (df_usa['STATE_NAME'] == 'Delaware') | (df_usa['STATE_NAME'] == 'Florida') | 
                  (df_usa['STATE_NAME'] == 'Georgia') | (df_usa['STATE_NAME'] == 'Illinois') | 
                  (df_usa['STATE_NAME'] == 'Indiana') | (df_usa['STATE_NAME'] == 'Kentucky') | 
                  (df_usa['STATE_NAME'] == 'Maine') | (df_usa['STATE_NAME'] == 'Maryland') | 
                  (df_usa['STATE_NAME'] == 'Massachusetts') | (df_usa['STATE_NAME'] == 'Michigan') | 
                  (df_usa['STATE_NAME'] == 'Missouri') | (df_usa['STATE_NAME'] == 'New Hampshire') | 
                  (df_usa['STATE_NAME'] == 'New Jersey') | (df_usa['STATE_NAME'] == 'New York') | 
                  (df_usa['STATE_NAME'] == 'North Carolina') | (df_usa['STATE_NAME'] == 'Ohio') | 
                  (df_usa['STATE_NAME'] == 'Pennsylvania') | (df_usa['STATE_NAME'] == 'Rhode Island') | 
                  (df_usa['STATE_NAME'] == 'South Carolina') | (df_usa['STATE_NAME'] == 'Tennessee') | 
                  (df_usa['STATE_NAME'] == 'Vermont') | (df_usa['STATE_NAME'] == 'Virginia') | 
                  (df_usa['STATE_NAME'] == 'West Virginia') | (df_usa['STATE_NAME'] == 'Wisconsin')]

df_usa_c = df_usa_c.reset_index(drop=True)

print(df_usa_c.shape)

df_usa_c.head(2)

In [ ]:
number_of_years = 5

# number_of_years = 3

In [ ]:
# predictor_variable = 'year'

predictor_variable = 'air_temp_degrees_celsius'

In [ ]:
def name_of_function(species, predictor_variable):

    df_species = df_merged[df_merged['species'] == species]
    
    df_species = df_species.drop(columns='air_temp_kelvin')

    print(df_species.shape)

    # print(df_species.head(2))
    
    # model_mixed = smf.mixedlm("arr_IAR_mean ~ air_temp_degrees_celsius", df_species, groups=df_species['lat'])

    # fit_mixed = model_mixed.fit()

    # print(fit_mixed.summary())
    
    model_ols = smf.ols(formula='arr_IAR_mean ~ air_temp_degrees_celsius', data=df_species)

    res_ols = model_ols.fit()
    
    print(res_ols.summary())

    df_cnt = df_species[['cell', 'year']].groupby(['cell']).count()

    df_cnt = df_cnt.reset_index()

    df_cnt = df_cnt.rename(columns={'year': 'number_of_years'})

    print(df_cnt.shape)

    # print(df_cnt.head(2))

    list_cells = list(df_cnt[df_cnt['number_of_years'] >= number_of_years]['cell'])
    print(len(list_cells))

    print(len(list(df_cnt[df_cnt['number_of_years'] >= 3]['cell'])))

    df_species = df_species[df_species['cell'].isin(list_cells)]
    print(df_species.shape)

    list_slopes = []
    list_intercepts = []
    list_r_squared = []
    list_p_values = []

    for cell in list_cells:

        df_cell = df_species[df_species['cell'] == cell]
        # print(df_cell.shape)

        model_ols = smf.ols(formula='arr_IAR_mean' + ' ~ ' + predictor_variable, data=df_cell)

        res_ols = model_ols.fit()

        # print(res_ols.summary())

        list_slopes.append(res_ols.params[1])

        list_intercepts.append(res_ols.params[0])

        list_r_squared.append(res_ols.rsquared)

        list_p_values.append(res_ols.pvalues[1])

        # ax = df_cell.plot.scatter(x=predictor_variable, y='arr_IAR_mean')

        # abline_plot(model_results=res_ols, ax=ax)

        # plt.show()

    df_results = pd.DataFrame(data={'cell': list_cells, 'slope': list_slopes, 'intercept': list_intercepts, 
                                    'r_squared': list_r_squared, 'p_value': list_p_values})
    print(df_results.shape)

    # print(df_results.head(2))

    # Quality control?

    df_lat_lon = df_t[['cell', 'lat', 'lon']].drop_duplicates()

    # print(df_lat_lon.shape)

    # print(df_lat_lon.head(2))

    df_results = df_results.merge(df_lat_lon, on='cell')

    df_results['scientific_name'] = species

    print(df_results.shape)

    # print(df_results.head(2))
    
    # Latitude

    predictor_variable = 'lat'

    model_ols = smf.ols(formula='slope' + ' ~ ' + predictor_variable, data=df_results)

    res_ols = model_ols.fit()

    slope_lat = res_ols.params[1]

    ax = df_results.plot.scatter(x=predictor_variable, y='slope')

    abline_plot(model_results=res_ols, ax=ax)

    plt.show()
    
    # Longitude

    # predictor_variable = 'lon'

    # model_ols = smf.ols(formula='slope' + ' ~ ' + predictor_variable, data=df_results)

    # res_ols = model_ols.fit()

    # print(res_ols.params[1])

    # ax = df_results.plot.scatter(x=predictor_variable, y='slope')

    # abline_plot(model_results=res_ols, ax=ax)

    # plt.show()
    
    # print(df_results['slope'].min())
    # print(np.floor(df_results['slope'].min()))
    # print(df_results['slope'].max())
    # print(np.ceil(df_results['slope'].max()))

    abs_floor_min_slope = np.absolute(np.floor(df_results['slope'].min()))

    abs_ceil_max_slope = np.absolute(np.ceil(df_results['slope'].max()))

    greatest_abs = max([abs_floor_min_slope, abs_ceil_max_slope])

    v_min = -greatest_abs

    v_max = greatest_abs

    norm = matplotlib.colors.Normalize(vmin=v_min, vmax=v_max)
    
    df_g = geopandas.GeoDataFrame(df_results, geometry=geopandas.points_from_xy(df_results['lon'], 
                                                                                df_results['lat']))
    print(df_g.shape)

    df_g.head(2)
    
    df_usa_c.plot(color='white', edgecolor='grey', figsize=(10, 10))

    sc = plt.scatter('lon', 'lat', s=1080, c='slope', alpha=0.8, cmap='RdBu_r', data=df_g, 
                     norm=norm, edgecolor='black', marker='s')

    cbar = plt.colorbar(sc, fraction=0.0241, pad=0.015)
    
    cbar.set_label('Sensitivity (days/°C)')

#     plt.xlabel('Longitude')

#     plt.ylabel('Latitude')

    size_small = 24
    size_medium = 24

    plt.rc('font', size=size_small)
    plt.rc('axes', titlesize=size_small)
    plt.rc('axes', labelsize=size_medium)
    plt.rc('xtick', labelsize=size_small)
    plt.rc('ytick', labelsize=size_small)
    
    if species == 'Hirundo_rustica':
        
        plt.savefig('20220621.png', bbox_inches='tight')

    plt.show()
    
    return df_results, slope_lat

In [ ]:
df_results_all_species = pd.DataFrame()

list_slopes_lat = []

for species in list_sci_names:

    print(species)

    df_results_species, slope_lat = name_of_function(species, predictor_variable)

    df_results_all_species = df_results_all_species.append(df_results_species)
    
    list_slopes_lat.append(slope_lat)

In [ ]:
df_results_all_species['scientific_name'] = df_results_all_species['scientific_name'].str.replace('_', ' ')
print(df_results_all_species.shape)

df_results_all_species.head(2)

In [ ]:
df_slopes_lat = pd.DataFrame(data={'scientific_name': list_sci_names, 'slope': list_slopes_lat})

df_slopes_lat['scientific_name'] = df_slopes_lat['scientific_name'].str.replace('_', ' ')

print(df_slopes_lat.shape)

df_slopes_lat.head(2)

In [ ]:
os.getcwd()

In [ ]:
path = '/Users/alvastrand/Library/Mobile Documents/com~apple~CloudDocs/Documents/OU/Research/data/output/'

os.chdir(path)

In [ ]:
os.getcwd()

In [ ]:
filename = 'aerial_insectivores_migration_distances_mean_weighted_body_masses.csv'

df_masses_dist = pd.read_csv(filename)
print(df_masses_dist.shape)

df_masses_dist.head(2)

In [ ]:
df_masses_dist = df_masses_dist[['scientific_name', 'common_name', 'body_mass', 'distance']]
print(df_masses_dist.shape)

df_masses_dist.head(2)

In [ ]:
print(plt.rcParams['lines.markersize'])

In [ ]:
def make_plot(df):

    df_results_all_species_masses_dist = df.merge(df_masses_dist, on='scientific_name')

    df_results_all_species_masses_dist['body_mass_mg'] = df_results_all_species_masses_dist['body_mass']*1000

    df_results_all_species_masses_dist['log_body_mass_mg'] = np.log(
        df_results_all_species_masses_dist['body_mass_mg'])

    color_values = matplotlib.cm.rainbow(
        np.linspace(0, 1, len(df_results_all_species_masses_dist['common_name'].unique())))

    colors = dict(zip(df_results_all_species_masses_dist['common_name'].unique(), color_values))

    field_name = 'log_body_mass_mg'

#     field_name = 'distance'

    fig, ax = plt.subplots()

    for species in list(df_results_all_species_masses_dist['common_name'].unique()):

        df_subset = df_results_all_species_masses_dist[df_results_all_species_masses_dist['common_name'] == species]

        df_subset.plot(ax=ax, kind='scatter', x=field_name, y='slope', label=species, 
                       color=colors[species], figsize=(10,10), s=50)

    plt.rc('font', size=24)
    plt.rc('axes', titlesize=24)
    plt.rc('axes', labelsize=24)
    plt.rc('xtick', labelsize=24)
    plt.rc('ytick', labelsize=24)
    plt.rc('legend', fontsize=11)

    plt.legend(loc='best')

    plt.axhline(y=0, color='grey', linestyle='--')

    if field_name == 'log_body_mass_mg':

        plt.xlabel('Natural logarithm of body mass (mg)')
    
    plt.ylabel('Sensitivity (days/°C)')
    
    plt.savefig('sensitivity_body_mass_20220621.png', bbox_inches='tight')

    plt.show()
    
    return df_results_all_species_masses_dist

In [ ]:
df_results_all_species_masses_dist = make_plot(df_results_all_species)
print(df_results_all_species_masses_dist.shape)

df_results_all_species_masses_dist.head(2)

In [ ]:
df_slopes_lat_masses_dist = make_plot(df_slopes_lat)
print(df_slopes_lat_masses_dist.shape)

df_slopes_lat_masses_dist.head(2)

In [ ]:
model_ols = smf.ols(formula='slope ~ body_mass', data=df_results_all_species_masses_dist)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_results_all_species_masses_dist.plot.scatter(x='body_mass', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
model_ols = smf.ols(formula='slope ~ log_body_mass_mg', data=df_results_all_species_masses_dist)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_results_all_species_masses_dist.plot.scatter(x='log_body_mass_mg', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
# model_ols = smf.ols(formula='slope ~ body_mass * distance', data=df_results_all_species_masses_dist)

# res_ols = model_ols.fit()

# print(res_ols.summary())

In [ ]:
model_ols = smf.ols(formula='slope ~ body_mass', data=df_slopes_lat_masses_dist)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_slopes_lat_masses_dist.plot.scatter(x='body_mass', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
# model_ols = smf.ols(formula='slope ~ body_mass * distance', data=df_slopes_lat_masses_dist)

# res_ols = model_ols.fit()

# print(res_ols.summary())

In [ ]:
df_results_all_species_masses_dist_speeds_means = df_results_all_species_masses_dist.merge(
    df_speeds_means, on='scientific_name')
print(df_results_all_species_masses_dist_speeds_means.shape)

df_results_all_species_masses_dist_speeds_means.head(2)

In [ ]:
model_ols = smf.ols(formula='slope ~ beta_gamma_mean', data=df_results_all_species_masses_dist_speeds_means)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_results_all_species_masses_dist_speeds_means.plot.scatter(x='beta_gamma_mean', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
model_ols = smf.ols(formula='slope ~ mean_arrival_day', data=df_results_all_species_masses_dist_speeds_means)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_results_all_species_masses_dist_speeds_means.plot.scatter(x='mean_arrival_day', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
model_ols = smf.ols(formula='slope ~ distance', data=df_results_all_species_masses_dist_speeds_means)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_results_all_species_masses_dist_speeds_means.plot.scatter(x='distance', y='slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
# model_ols = smf.ols(formula='slope ~  body_mass * distance * beta_gamma_mean * mean_arrival_day', 
#                     data=df_results_all_species_masses_dist_speeds_means)

# res_ols = model_ols.fit()

# print(res_ols.summary())

In [ ]:
df_slopes_means = df_results_all_species_masses_dist[['scientific_name', 'slope']].groupby(['scientific_name']).mean()

df_slopes_means = df_slopes_means.reset_index()

df_slopes_means = df_slopes_means.rename(columns={'slope': 'mean_slope'})

print(df_slopes_means.shape)

df_slopes_means.head(2)

In [ ]:
df_slopes_medians = df_results_all_species_masses_dist[
    ['scientific_name', 'slope']].groupby(['scientific_name']).median()

df_slopes_medians = df_slopes_medians.reset_index()

df_slopes_medians = df_slopes_medians.rename(columns={'slope': 'median_slope'})

print(df_slopes_medians.shape)

df_slopes_medians.head(2)

In [ ]:
df_slopes_means_medians = df_slopes_means.merge(df_slopes_medians, on='scientific_name')
print(df_slopes_means_medians.shape)

df_slopes_means_medians.head(2)

In [ ]:
df_slopes_means_medians = df_slopes_means_medians.merge(df_masses_dist, on='scientific_name')
print(df_slopes_means_medians.shape)

df_slopes_means_medians.head(2)

In [ ]:
df_slopes_means_medians = df_slopes_means_medians.merge(df_speeds_means, on='scientific_name')
print(df_slopes_means_medians.shape)

df_slopes_means_medians.head(2)

In [ ]:
model_ols = smf.ols(formula='mean_slope ~ body_mass', data=df_slopes_means_medians)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_slopes_means_medians.plot.scatter(x='body_mass', y='mean_slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
model_ols = smf.ols(formula='median_slope ~ body_mass', data=df_slopes_means_medians)

res_ols = model_ols.fit()

print(res_ols.summary())

ax = df_slopes_means_medians.plot.scatter(x='body_mass', y='median_slope')

abline_plot(model_results=res_ols, ax=ax)

plt.show()

In [ ]:
model_ols = smf.ols(formula='mean_slope ~ body_mass * distance', data=df_slopes_means_medians)

res_ols = model_ols.fit()

print(res_ols.summary())

In [ ]:
filename = 'all_obligate_aerial_insectivores_ebirdst_weighted_mean_body_masses.csv'

df_species_codes = pd.read_csv(filename)
print(df_species_codes.shape)

df_species_codes.head(2)

In [ ]:
df_results_all_species_masses_dist_codes = df_results_all_species_masses_dist.merge(
    df_species_codes, on=['scientific_name', 'common_name', 'body_mass'])

df_results_all_species_masses_dist_codes = df_results_all_species_masses_dist_codes.sort_values(by='body_mass')

df_results_all_species_masses_dist_codes = df_results_all_species_masses_dist_codes.reset_index(drop=True)

print(df_results_all_species_masses_dist_codes.shape)

# print(df_results_all_species_masses_dist_codes.head(2))

cnt = 0

for species in list(df_results_all_species_masses_dist_codes['species_code'].unique()):
    
#     print(species)

    df_species_slopes = df_results_all_species_masses_dist_codes[
        df_results_all_species_masses_dist_codes['species_code'] == species]
    
    df_species_slopes = df_species_slopes[['slope']]
    
    df_species_slopes = df_species_slopes.rename(columns={'slope': species})
    
    df_species_slopes = df_species_slopes.reset_index(drop=True)

#     print(df_species_slopes.shape)
    
    if cnt == 0:

        df_species_slopes_merged = df_species_slopes

    else:

        df_species_slopes_merged = df_species_slopes_merged.merge(
            df_species_slopes, how='outer', left_index=True, right_index=True)

    cnt += 1
    
print(df_species_slopes_merged.shape)
    
# print(df_species_slopes_merged.head(2))

df_species_slopes_merged.boxplot(column=list(df_species_slopes_merged.columns), figsize=(10,6))

size_small = 14
size_medium = 14
size_large = 16

plt.rc('font', size=size_small)
plt.rc('axes', titlesize=size_small)
plt.rc('axes', labelsize=size_small)
plt.rc('xtick', labelsize=size_small)
plt.rc('ytick', labelsize=size_small)

plt.show()